# Sparse Operation Kit #
---
This notebook introduces what is sparse operation kit and how to use it to accerlerate the recommander system's training process.

Sparse Opertion Kit (hereafter SOK) is a toolkit aiming at wrapping those effecient algorithms / implementations used in recommendation scenarios, which includes many sparse operations, into a user-friendly library. When user wants to leverage those GPU-accerlerated algorithms to speed up their application, they can quickly start from this Python toolkit. 

## Features ##
- Embedding Algorithms:
    - HugeCTR::Distributed, which supports combiner={Mean, Sum}
- Synchronized Training:
    - Single-node, multiple GPUs
    - Multi-node, multiple GPUs
- Data-Parallel Inputs
    - No further burden to datareader except common data preprocessing.
- Deep Learning Framework compatibility:
    - TensorFlow
        - version
            - 2.x
        - optimizers
            - Adam
            - SGD

## Menu ##
1. **Installation**
2. **Single-node, Multi-GPUs synchronized training**
3. **Multi-node, Multi-GPUs synchronized training**

### Installation ###
+ **Requirements**
    - TensorFlow 2.x


+ **Get SOK from NGC** <br>
The SparseOperationKit is preinstalled in the [Merlin Tensorflow Training Container](https://ngc.nvidia.com/catalog/containers/nvidia:merlin:merlin-tensorflow-training): `nvcr.io/nvidia/merlin/merlin-tensorflow-training:21.11`. <br>
You can check the existence of required libraries by running the following Python code after launching this container.
```shell
$ python3 -c "import sparse_operation_kit as sok"
```

+ **Build SOK from Souce Code** <br>
If you want to build SparseOperationKit from the souce code instead of using the NGC container, please refer to the [Setup development environment](../docs/hugectr_contributor_guide.md#build-sparse-operation-kit-sok-from-source-code).

### Single-node, Multi-GPUs synchronized training ###

Firstly, specify hyper parameters.

In [1]:
%reset -f

args = dict()

args["gpu_num"] = 8                               # the number of available GPUs
args["iter_num"] = 50                             # the number of training iteration
args["max_vocabulary_size_per_gpu"] = 1024
args["slot_num"] = 10                             # the number of feature fields in this embedding layer
args["max_nnz"] = 4                               # the maximum number of valid features in each slot
args["embedding_vec_size"] = 4                    # the dimension of embedding vectors
args["combiner"] = "mean"                         # the reduction combiner used intra slots, it can be [mean, sum]
args["global_batch_size"] = 65536                 # the globally batchsize for all GPUs
args["optimizer"] = "plugin_adam"                 # the optimizer used for training, it can be [plugin_adam, adam, sgd]

Secondly, import the used modules.

In [2]:
import sys, os, json
sys.path.append("../")
import sparse_operation_kit as sok
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, range(args["gpu_num"])))
import numpy as np

[INFO]: sparse_operation_kit is imported


Thirdly, define a DNN model using TensorFlow.

In [3]:
class TfDemo(tf.keras.models.Model):
    def __init__(self, 
                 init_tensors, 
                 combiner, 
                 global_batch_size,
                 slot_num, 
                 embedding_vec_size,
                 **kwargs):
        super(TfDemo, self).__init__(**kwargs)
        self.combiner = combiner
        self.global_batch_size = global_batch_size
        self.slot_num = slot_num
        self.embedding_vec_size = embedding_vec_size

        self.init_tensors = init_tensors
        self.params = tf.Variable(initial_value=tf.concat(self.init_tensors, axis=0))

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize * slot_num, embedding_vec_size]
        embedding_vector = tf.nn.embedding_lookup_sparse(params=self.params, sp_ids=inputs,
                                                        sp_weights=None, combiner=self.combiner)

        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[self.global_batch_size, self.slot_num * self.embedding_vec_size])
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

Fourthly, define the same DNN model using SOK.

In [4]:
class SOKDemo(tf.keras.models.Model):
    def __init__(self,
                 combiner,
                 max_vocabulary_size_per_gpu,
                 slot_num,
                 max_nnz,
                 embedding_vec_size, 
                 **kwargs):
        super(SOKDemo, self).__init__(**kwargs)

        self.combiner = combiner
        self.max_vocabulary_size_per_gpu = max_vocabulary_size_per_gpu
        self.slot_num = slot_num
        self.max_nnz = max_nnz
        self.embedding_vec_size = embedding_vec_size

        self.embedding_layer = sok.DistributedEmbedding(combiner=self.combiner,
                                                           max_vocabulary_size_per_gpu=self.max_vocabulary_size_per_gpu,
                                                           embedding_vec_size=self.embedding_vec_size,
                                                           slot_num=self.slot_num,
                                                           max_nnz=self.max_nnz)

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize, slot_num, embedding_vec_size]
        embedding_vector = self.embedding_layer(inputs, training=training)
        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[-1, self.slot_num * self.embedding_vec_size])
        # [batchsize, 1]
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

Fifthly, generate synthetic dataset and initial values that is used to initialize embedding parameters.

In [5]:
# import utility python script
sys.path.append("../unit_test/test_scripts/")
import utils

In [6]:
# -1 is used to represent the invalid keys
random_samples = utils.generate_random_samples(num_of_samples=args["global_batch_size"] * args["iter_num"],
                                               vocabulary_size=args["gpu_num"] * args["max_vocabulary_size_per_gpu"],
                                               slot_num=args["slot_num"],
                                               max_nnz=args["max_nnz"])

[INFO]: begin to generate random samples
[INFO]: generated random samples


In [7]:
# check ramdom_samples
random_samples

(array([[[  38,   -1,   -1,   -1],
         [ 819,   -1,   -1,   -1],
         [2444, 2431,   -1,   -1],
         ...,
         [6389, 5973,   -1,   -1],
         [  -1,   -1,   -1,   -1],
         [7910, 7850,   -1,   -1]],
 
        [[ 676,  607,  573,   -1],
         [1631, 1563,   -1,   -1],
         [1733, 1699,   -1,   -1],
         ...,
         [6459, 5950,   -1,   -1],
         [7234, 6724,   -1,   -1],
         [8163,   -1,   -1,   -1]],
 
        [[ 438,  283,   -1,   -1],
         [1212, 1057,  902,   -1],
         [1831,   -1,   -1,   -1],
         ...,
         [6414, 6336,   -1,   -1],
         [6863,   -1,   -1,   -1],
         [7469,   -1,   -1,   -1]],
 
        ...,
 
        [[ 688,  618,   -1,   -1],
         [1257,   -1,   -1,   -1],
         [1975, 1897, 1647,   -1],
         ...,
         [6439, 6337,   -1,   -1],
         [7309, 6642,   -1,   -1],
         [7665, 7614, 7512,   -1]],
 
        [[ 505,  454,   -1,   -1],
         [1527, 1476, 1426,   -1],
       

In [8]:
# generate initial value for embedding parameters
init_tensors = utils.get_ones_tensor(max_vocab_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                                     embedding_vec_size=args["embedding_vec_size"],
                                     num=args["gpu_num"])

In [9]:
# check init_tensors
init_tensors

[array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32),
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1

Sixly, define training loop for TensorFlow and SOK

In [10]:
def test_tf_demo(args, init_tensors, *random_samples):
    dataset = utils.tf_dataset(*random_samples, batchsize=args["global_batch_size"], to_sparse_tensor=True, repeat=1)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    tf_demo = TfDemo(init_tensors, args["combiner"], args["global_batch_size"], 
                     args["slot_num"], args["embedding_vec_size"])

    optimizer = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = tf_demo(inputs, training=True)
            loss = loss_fn(labels, logit)
        grads = tape.gradient(loss, tf_demo.trainable_variables)
        optimizer.apply_gradients(zip(grads, tf_demo.trainable_variables))
        return logit, embedding_vector

    tf_results = list()

    for i, (sparse_tensors, labels) in enumerate(dataset):
        print("-"*30, str(i), "-"*30)
        logit, embedding_vector = _train_step(sparse_tensors, labels)
        print("[INFO]: embedding_vector:\n", embedding_vector)
        tf_results.append(embedding_vector)

        # FIXME: because plugin sleepd, here is only used for 
        # simulate the same DNN structure. 
        import time
        time.sleep(0.2) # seconds

    return tf_results

In [11]:
def test_sok_demo(args, init_tensors, *random_samples):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        result = sok.Init(global_batch_size=args["global_batch_size"])

        plugin_demo = SOKDemo(combiner=args["combiner"], 
                                 max_vocabulary_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                                 slot_num=args["slot_num"], max_nnz=args["max_nnz"],
                                 embedding_vec_size=args["embedding_vec_size"])

        emb_opt = utils.get_embedding_optimizer(args["optimizer"])(learning_rate=0.1)
        dense_opt = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)

    plugin_saver = sok.Saver()

    plugin_saver.load_embedding_values(plugin_demo.embedding_layer.embedding_variable, init_tensors)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    def _replica_loss(labels, logits):
        loss = loss_fn(labels, logits)
        return tf.nn.compute_average_loss(loss, global_batch_size=args["global_batch_size"])

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = plugin_demo(inputs, training=True)
            loss = _replica_loss(labels, logit)
        embedding_variables, other_variable = sok.split_embedding_variable_from_others(plugin_demo.trainable_variables)
        grads, emb_grads = tape.gradient(loss, [other_variable, embedding_variables])
        if 'plugin' not in args["optimizer"]:
            with sok.OptimizerScope(embedding_variables):
                emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                        experimental_aggregate_gradients=False)
        else:
            emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                    experimental_aggregate_gradients=False)
        dense_opt.apply_gradients(zip(grads, other_variable))
        return logit, embedding_vector

    sok_results = list()

    def _dataset_fn(input_context):
        replica_batch_size = input_context.get_per_replica_batch_size(args["global_batch_size"])
        dataset = utils.tf_dataset(*random_samples, batchsize=replica_batch_size, to_sparse_tensor=True, repeat=1)
        dataset = dataset.shard(input_context.num_input_pipelines, input_context.input_pipeline_id)
        return dataset

    dataset = strategy.distribute_datasets_from_function(_dataset_fn)
    
    for i, (sparse_tensors, replica_labels) in enumerate(dataset):
        print("-" * 30, "step ", str(i), "-" * 30)
        logit, embedding_vector = strategy.run(_train_step, args=(sparse_tensors, replica_labels))
        print("[INFO]: embedding_vector\n", embedding_vector)
        sok_results.append(embedding_vector)

        # FIXME: when the forward computation is too fast, there
        # may exist some conficts with datareader, which cause the program hang.
        import time
        time.sleep(0.2) # seconds

    return sok_results

Sevenly, start training process. Compare whether the embedding vectors obtained from TensorFlow and SOK are consistent in all iterations.

In [12]:
# train TensorFlow Demo Model, this command will print each iteration's embedding vector
tf_results = test_tf_demo(args, init_tensors, *random_samples)

------------------------------ 0 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(65536, 40), dtype=float32)
------------------------------ 1 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[0.9004926  0.9004926  0.9004926  ... 0.90051043 0.90051043 0.90051043]
 [0.9005401  0.9005401  0.9005401  ... 0.90051115 0.90051115 0.90051115]
 [0.9005135  0.9005135  0.9005135  ... 0.90062284 0.90062284 0.90062284]
 ...
 [0.90055627 0.90055627 0.90055627 ... 0.9005     0.9005     0.9005    ]
 [0.9004842  0.9004842  0.9004842  ... 0.90050197 0.90050197 0.90050197]
 [0.90063393 0.90063393 0.90063393 ... 0.9005594  0.9005594  0.9005594 ]], shape=(65536, 40), dtype=float32)
------------------------------ 2 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[0.8013395  0.8013395  

------------------------------ 15 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.17062765 -0.17062765 -0.17062765 ... -0.17748016 -0.17748016
  -0.17748016]
 [-0.18568465 -0.18568465 -0.18568465 ... -0.16705126 -0.16705126
  -0.16705126]
 [-0.15869047 -0.15869047 -0.15869047 ... -0.17683107 -0.17683107
  -0.17683107]
 ...
 [-0.16965656 -0.16965656 -0.16965656 ... -0.18408883 -0.18408883
  -0.18408883]
 [-0.16662782 -0.16662782 -0.16662782 ... -0.17666113 -0.17666113
  -0.17666113]
 [-0.15216029 -0.15216029 -0.15216029 ... -0.17087948 -0.17087948
  -0.17087948]], shape=(65536, 40), dtype=float32)
------------------------------ 16 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.24626109 -0.24626109 -0.24626109 ... -0.21986282 -0.21986282
  -0.21986282]
 [-0.20474827 -0.20474827 -0.20474827 ... -0.20359749 -0.20359749
  -0.20359749]
 [-0.20505694 -0.20505694 -0.20505694 ... -0.24400447 -0.24400447
  -0.24400447]
 ...
 [-0.21973535 -0.21

------------------------------ 29 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.19414708 -0.19414708 -0.19414708 ... -0.16186176 -0.16186176
  -0.16186176]
 [-0.18264094 -0.18264094 -0.18264094 ... -0.18946269 -0.18946269
  -0.18946269]
 [-0.18857247 -0.18857247 -0.18857247 ... -0.19120839 -0.19120839
  -0.19120839]
 ...
 [-0.1658213  -0.1658213  -0.1658213  ... -0.19272026 -0.19272026
  -0.19272026]
 [-0.1939144  -0.1939144  -0.1939144  ... -0.1559137  -0.1559137
  -0.1559137 ]
 [-0.16430692 -0.16430692 -0.16430692 ... -0.17493583 -0.17493583
  -0.17493583]], shape=(65536, 40), dtype=float32)
------------------------------ 30 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.17045125 -0.17045125 -0.17045125 ... -0.15462823 -0.15462823
  -0.15462823]
 [-0.17679203 -0.17679203 -0.17679203 ... -0.16658922 -0.16658922
  -0.16658922]
 [-0.15218404 -0.15218404 -0.15218404 ... -0.15486553 -0.15486553
  -0.15486553]
 ...
 [-0.15468912 -0.154

------------------------------ 43 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.2105065  -0.2105065  -0.2105065  ... -0.18041237 -0.18041237
  -0.18041237]
 [-0.20118019 -0.20118019 -0.20118019 ... -0.26570445 -0.26570445
  -0.26570445]
 [-0.20779352 -0.20779352 -0.20779352 ... -0.18782339 -0.18782339
  -0.18782339]
 ...
 [-0.18462594 -0.18462594 -0.18462594 ... -0.19564806 -0.19564806
  -0.19564806]
 [-0.22160469 -0.22160469 -0.22160469 ... -0.18537885 -0.18537885
  -0.18537885]
 [-0.20659022 -0.20659022 -0.20659022 ... -0.22612928 -0.22612928
  -0.22612928]], shape=(65536, 40), dtype=float32)
------------------------------ 44 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.2109933  -0.2109933  -0.2109933  ... -0.21877787 -0.21877787
  -0.21877787]
 [-0.21374518 -0.21374518 -0.21374518 ... -0.16920957 -0.16920957
  -0.16920957]
 [-0.23200426 -0.23200426 -0.23200426 ... -0.20374995 -0.20374995
  -0.20374995]
 ...
 [-0.22763464 -0.22

In [13]:
# train SOK Demo Model, this command will print each iteration's embedding vector 
sok_results = test_sok_demo(args, init_tensors, *random_samples)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
You are using the plugin with MirroredStrategy.
------------------------------ step  0 ------------------------------
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[1. 1. 1. ..

------------------------------ step  3 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[0.7060577  0.7060577  0.7060577  ... 0.70395017 0.70395017 0.70395017]
 [0.7036203  0.7036203  0.7036203  ... 0.7027384  0.7027384  0.7027384 ]
 [0.7021664  0.7021664  0.7021664  ... 0.7022897  0.7022897  0.7022897 ]
 ...
 [0.7051804  0.7051804  0.7051804  ... 0.7032761  0.7032761  0.7032761 ]
 [0.70289946 0.70289946 0.70289946 ... 0.70413184 0.70413184 0.70413184]
 [0.7064773  0.7064773  0.7064773  ... 0.703575   0.703575   0.703575  ]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[0.7029407  0.7029407  0.7029407  ... 0.70313156 0.70313156 0.70313156]
 [0.70317686 0.70317686 0.70317686 ... 0.70624536 0.70624536 0.70624536]
 [0.7043319  0.7043319  0.7043319  ... 0.7035043  0.7035043  0.7035043 ]
 ...
 [0.70132977 0.70132977 0.70132977 ... 0.70977783 0.70977783 0.70977783]
 [0.7031638  0.7031638  0.7031638  ... 0.7029564  0.7029564  0.7029564 ]
 [0.703057

------------------------------ step  6 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[0.4105538  0.4105538  0.4105538  ... 0.42588177 0.42588177 0.42588177]
 [0.4197083  0.4197083  0.4197083  ... 0.41922885 0.41922885 0.41922885]
 [0.41254905 0.41254905 0.41254905 ... 0.41198918 0.41198918 0.41198918]
 ...
 [0.42641217 0.42641217 0.42641217 ... 0.41832227 0.41832227 0.41832227]
 [0.42001283 0.42001283 0.42001283 ... 0.413566   0.413566   0.413566  ]
 [0.4191956  0.4191956  0.4191956  ... 0.4227935  0.4227935  0.4227935 ]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[0.4204505  0.4204505  0.4204505  ... 0.4150052  0.4150052  0.4150052 ]
 [0.42168778 0.42168778 0.42168778 ... 0.41749144 0.41749144 0.41749144]
 [0.41580355 0.41580355 0.41580355 ... 0.42963973 0.42963973 0.42963973]
 ...
 [0.42314714 0.42314714 0.42314714 ... 0.4265498  0.4265498  0.4265498 ]
 [0.41508466 0.41508466 0.41508466 ... 0.41742906 0.41742906 0.41742906]
 [0.418328

------------------------------ step  9 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[0.1579948  0.1579948  0.1579948  ... 0.17363048 0.17363048 0.17363048]
 [0.14760572 0.14760572 0.14760572 ... 0.15634492 0.15634492 0.15634492]
 [0.15677038 0.15677038 0.15677038 ... 0.16316155 0.16316155 0.16316155]
 ...
 [0.16873369 0.16873369 0.16873369 ... 0.15639687 0.15639687 0.15639687]
 [0.16345012 0.16345012 0.16345012 ... 0.1576001  0.1576001  0.1576001 ]
 [0.16045508 0.16045508 0.16045508 ... 0.15251702 0.15251702 0.15251702]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[0.15678135 0.15678135 0.15678135 ... 0.16477747 0.16477747 0.16477747]
 [0.16192615 0.16192615 0.16192615 ... 0.16698089 0.16698089 0.16698089]
 [0.15099943 0.15099943 0.15099943 ... 0.17474762 0.17474762 0.17474762]
 ...
 [0.16206431 0.16206431 0.16206431 ... 0.16256058 0.16256058 0.16256058]
 [0.1687254  0.1687254  0.1687254  ... 0.16451517 0.16451517 0.16451517]
 [0.154466

------------------------------ step  12 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.01844369 -0.01844369 -0.01844369 ... -0.03240526 -0.03240526
  -0.03240526]
 [-0.03888315 -0.03888315 -0.03888315 ... -0.02606202 -0.02606202
  -0.02606202]
 [-0.02816605 -0.02816605 -0.02816605 ... -0.02493167 -0.02493167
  -0.02493167]
 ...
 [-0.02413027 -0.02413027 -0.02413027 ... -0.04944995 -0.04944995
  -0.04944995]
 [-0.03621512 -0.03621512 -0.03621512 ... -0.01311048 -0.01311048
  -0.01311048]
 [-0.03537244 -0.03537244 -0.03537244 ... -0.03908113 -0.03908113
  -0.03908113]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.02457406 -0.02457406 -0.02457406 ... -0.00881258 -0.00881258
  -0.00881258]
 [-0.02927418 -0.02927418 -0.02927418 ... -0.03563254 -0.03563254
  -0.03563254]
 [-0.02485804 -0.02485804 -0.02485804 ... -0.02413274 -0.02413274
  -0.02413274]
 ...
 [-0.02219195 -0.02219195 -0.02219195 ... -0.02059275 -0.02059275
  -0.02059275]
 [

...

------------------------------ step  40 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.14578068 -0.14578068 -0.14578068 ... -0.20067462 -0.20067462
  -0.20067462]
 [-0.13057902 -0.13057902 -0.13057902 ... -0.19506142 -0.19506142
  -0.19506142]
 [-0.20201227 -0.20201227 -0.20201227 ... -0.16477202 -0.16477202
  -0.16477202]
 ...
 [-0.14560255 -0.14560255 -0.14560255 ... -0.16252151 -0.16252151
  -0.16252151]
 [-0.16606383 -0.16606383 -0.16606383 ... -0.17239738 -0.17239738
  -0.17239738]
 [-0.19666556 -0.19666556 -0.19666556 ... -0.15619218 -0.15619218
  -0.15619218]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.17178753 -0.17178753 -0.17178753 ... -0.17795643 -0.17795643
  -0.17795643]
 [-0.19007666 -0.19007666 -0.19007666 ... -0.19053559 -0.19053559
  -0.19053559]
 [-0.14407048 -0.14407048 -0.14407048 ... -0.14322627 -0.14322627
  -0.14322627]
 ...
 [-0.20170252 -0.20170252 -0.20170252 ... -0.16819994 -0.16819994
  -0.16819994]
 [

------------------------------ step  42 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.19167188 -0.19167188 -0.19167188 ... -0.26264137 -0.26264137
  -0.26264137]
 [-0.21755669 -0.21755669 -0.21755669 ... -0.2098138  -0.2098138
  -0.2098138 ]
 [-0.19374077 -0.19374077 -0.19374077 ... -0.20081697 -0.20081697
  -0.20081697]
 ...
 [-0.18769276 -0.18769276 -0.18769276 ... -0.20685501 -0.20685501
  -0.20685501]
 [-0.17655183 -0.17655183 -0.17655183 ... -0.19016416 -0.19016416
  -0.19016416]
 [-0.15241548 -0.15241548 -0.15241548 ... -0.19887695 -0.19887695
  -0.19887695]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.1931665  -0.1931665  -0.1931665  ... -0.13349657 -0.13349657
  -0.13349657]
 [-0.19947068 -0.19947068 -0.19947068 ... -0.19171639 -0.19171639
  -0.19171639]
 [-0.22939122 -0.22939122 -0.22939122 ... -0.2026543  -0.2026543
  -0.2026543 ]
 ...
 [-0.19573733 -0.19573733 -0.19573733 ... -0.21818948 -0.21818948
  -0.21818948]
 [-0

------------------------------ step  44 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.2109896  -0.2109896  -0.2109896  ... -0.21877484 -0.21877484
  -0.21877484]
 [-0.21374257 -0.21374257 -0.21374257 ... -0.16920632 -0.16920632
  -0.16920632]
 [-0.23200105 -0.23200105 -0.23200105 ... -0.2037467  -0.2037467
  -0.2037467 ]
 ...
 [-0.20910196 -0.20910196 -0.20910196 ... -0.1786393  -0.1786393
  -0.1786393 ]
 [-0.19057699 -0.19057699 -0.19057699 ... -0.17450081 -0.17450081
  -0.17450081]
 [-0.24734297 -0.24734297 -0.24734297 ... -0.22391725 -0.22391725
  -0.22391725]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.21556352 -0.21556352 -0.21556352 ... -0.20855372 -0.20855372
  -0.20855372]
 [-0.22166187 -0.22166187 -0.22166187 ... -0.18645974 -0.18645974
  -0.18645974]
 [-0.26165748 -0.26165748 -0.26165748 ... -0.20556459 -0.20556459
  -0.20556459]
 ...
 [-0.22729544 -0.22729544 -0.22729544 ... -0.26249874 -0.26249874
  -0.26249874]
 [-0

------------------------------ step  46 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.26487082 -0.26487082 -0.26487082 ... -0.23970175 -0.23970175
  -0.23970175]
 [-0.2404138  -0.2404138  -0.2404138  ... -0.26857358 -0.26857358
  -0.26857358]
 [-0.24418813 -0.24418813 -0.24418813 ... -0.22789705 -0.22789705
  -0.22789705]
 ...
 [-0.23508665 -0.23508665 -0.23508665 ... -0.23829213 -0.23829213
  -0.23829213]
 [-0.22368917 -0.22368917 -0.22368917 ... -0.2507833  -0.2507833
  -0.2507833 ]
 [-0.2787284  -0.2787284  -0.2787284  ... -0.21240795 -0.21240795
  -0.21240795]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.28410178 -0.28410178 -0.28410178 ... -0.27136627 -0.27136627
  -0.27136627]
 [-0.24789992 -0.24789992 -0.24789992 ... -0.2731675  -0.2731675
  -0.2731675 ]
 [-0.2615593  -0.2615593  -0.2615593  ... -0.2276803  -0.2276803
  -0.2276803 ]
 ...
 [-0.2253224  -0.2253224  -0.2253224  ... -0.24861962 -0.24861962
  -0.24861962]
 [-0.

------------------------------ step  48 ------------------------------
[INFO]: embedding_vector
 PerReplica:{
  0: tf.Tensor(
[[-0.22812551 -0.22812551 -0.22812551 ... -0.2713868  -0.2713868
  -0.2713868 ]
 [-0.2600761  -0.2600761  -0.2600761  ... -0.25804752 -0.25804752
  -0.25804752]
 [-0.22659056 -0.22659056 -0.22659056 ... -0.24469978 -0.24469978
  -0.24469978]
 ...
 [-0.21432105 -0.21432105 -0.21432105 ... -0.23977128 -0.23977128
  -0.23977128]
 [-0.28816798 -0.28816798 -0.28816798 ... -0.26758283 -0.26758283
  -0.26758283]
 [-0.20406762 -0.20406762 -0.20406762 ... -0.2416647  -0.2416647
  -0.2416647 ]], shape=(8192, 40), dtype=float32),
  1: tf.Tensor(
[[-0.22131589 -0.22131589 -0.22131589 ... -0.25021574 -0.25021574
  -0.25021574]
 [-0.21980968 -0.21980968 -0.21980968 ... -0.24015644 -0.24015644
  -0.24015644]
 [-0.25869566 -0.25869566 -0.25869566 ... -0.24941725 -0.24941725
  -0.24941725]
 ...
 [-0.23439011 -0.23439011 -0.23439011 ... -0.22400156 -0.22400156
  -0.22400156]
 [-0

Finally, check the consistency of the embedding vectors obtained from TensorFlow ans SOK.

In [14]:
if (len(sok_results) != len(tf_results)):
    raise ValueError("The length of sok results is not equal to that of TensorFlow.")
if (len(tf_results) != args["iter_num"]):
    raise ValueError("The length of embedding vectors: %d is not equal to iteration number: %d."
                    %(len(tf_results), args["iter_num"]))
    
for i, sok_vector in enumerate(sok_results):
    if args["gpu_num"] != 1:
        sok_vector = tf.stack(sok_vector.values, axis=0)
    tf.debugging.assert_near(tf.reshape(sok_vector,
                                        shape=[-1, tf.shape(sok_vector)[-1]]),
                             tf_results[i],
                             atol=1e-4,
                             rtol=1e-4,
                             message="The embedding vectors obtained from TF and SOK vary in iteration: %d" %i)
    
# if no exception, then the embedding vectors for all iterations are consistent.
print(("\n[INFO]: With MirroredStrategy, when %d GPUs are used, the embedding vectors obtained from TensorFlow" 
       "and SOK are consistent for %d iterations") 
      %(args["gpu_num"], args["iter_num"]))


[INFO]: With MirroredStrategy, when 8 GPUs are used, the embedding vectors obtained from TensorFlowand SOK are consistent for 50 iterations


### Multi-node, Multi-GPUs synchronized training ###

**The jupyter notebook must be restarted!!**

Firstly, specify hyper parameters

In [1]:
%reset -f

args = dict()

args["iter_num"] = 50                             # the number of training iteration
args["max_vocabulary_size_per_gpu"] = 1024
args["slot_num"] = 10                             # the number of feature fields in this embedding layer
args["max_nnz"] = 4                               # the maximum number of valid features in each slot
args["embedding_vec_size"] = 4                    # the dimension of embedding vectors
args["combiner"] = "mean"                         # the reduction combiner used intra slots, it can be [mean, sum]
args["global_batch_size"] = 65536                 # the globally batchsize for all GPUs
args["optimizer"] = "plugin_adam"                 # the optimizer used for training, it can be [plugin_adam, adam, sgd]
args["ips"] = ["localhost", "localhost"]          # specify the ip addr of each node. Here we use different GPUs to 
                                                  # simulate multi-node with single-node    
args["worker_num"] = len(args["ips"])             # the number of workers in synchronized training

In [2]:
import sys, os, json
sys.path.append("../")
import sparse_operation_kit as sok
import tensorflow as tf
import numpy as np
# import utility python script
sys.path.append("../unit_test/test_scripts/")
import utils

[INFO]: sparse_operation_kit is imported


In [3]:
total_gpu_num = utils.get_local_gpu_count()
print("[INFO]: There are %d GPUs in total" %total_gpu_num)
if (total_gpu_num % args["worker_num"] != 0):
    raise RuntimeError("total_gpu_num:%d is not divisible by workers_num: %d" %(total_gpu_num, args["worker_num"]))
    
per_worker_gpu_num = total_gpu_num // args["worker_num"]
args["local_gpu_num"] = per_worker_gpu_num # the number of avaiable GPUs in each process

[INFO]: There are 16 GPUs in total


Secondly, define DNN model using Tensorflow and SOK.

In [4]:
class TfDemo(tf.keras.models.Model):
    def __init__(self, 
                 init_tensors, 
                 combiner, 
                 global_batch_size,
                 slot_num, 
                 embedding_vec_size,
                 **kwargs):
        super(TfDemo, self).__init__(**kwargs)
        self.combiner = combiner
        self.global_batch_size = global_batch_size
        self.slot_num = slot_num
        self.embedding_vec_size = embedding_vec_size

        self.init_tensors = init_tensors
        self.params = tf.Variable(initial_value=tf.concat(self.init_tensors, axis=0))

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize * slot_num, embedding_vec_size]
        embedding_vector = tf.nn.embedding_lookup_sparse(params=self.params, sp_ids=inputs,
                                                        sp_weights=None, combiner=self.combiner)

        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[self.global_batch_size, self.slot_num * self.embedding_vec_size])
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

In [5]:
class SOKDemo(tf.keras.models.Model):
    def __init__(self,
                 combiner,
                 max_vocabulary_size_per_gpu,
                 slot_num,
                 max_nnz,
                 embedding_vec_size, 
                 **kwargs):
        super(SOKDemo, self).__init__(**kwargs)

        self.combiner = combiner
        self.max_vocabulary_size_per_gpu = max_vocabulary_size_per_gpu
        self.slot_num = slot_num
        self.max_nnz = max_nnz
        self.embedding_vec_size = embedding_vec_size

        self.embedding_layer = sok.DistributedEmbedding(combiner=self.combiner,
                                                           max_vocabulary_size_per_gpu=self.max_vocabulary_size_per_gpu,
                                                           embedding_vec_size=self.embedding_vec_size,
                                                           slot_num=self.slot_num,
                                                           max_nnz=self.max_nnz)

        self.dense_layer = tf.keras.layers.Dense(units=1, activation=None,
                                                 kernel_initializer="ones",
                                                 bias_initializer="zeros")

    def call(self, inputs, training=True):
        # [batchsize, slot_num, embedding_vec_size]
        embedding_vector = self.embedding_layer(inputs, training=training)
        # [batchsize, slot_num * embedding_vec_size]
        embedding_vector = tf.reshape(embedding_vector, shape=[-1, self.slot_num * self.embedding_vec_size])
        # [batchsize, 1]
        logit = self.dense_layer(embedding_vector)
        return logit, embedding_vector

In [6]:
def test_tf_demo(args, init_tensors, *random_samples):
    dataset = utils.tf_dataset(*random_samples, batchsize=args["global_batch_size"], to_sparse_tensor=True, repeat=1)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    tf_demo = TfDemo(init_tensors, args["combiner"], args["global_batch_size"], 
                     args["slot_num"], args["embedding_vec_size"])

    optimizer = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = tf_demo(inputs, training=True)
            loss = loss_fn(labels, logit)
        grads = tape.gradient(loss, tf_demo.trainable_variables)
        optimizer.apply_gradients(zip(grads, tf_demo.trainable_variables))
        return logit, embedding_vector

    tf_results = list()

    for i, (sparse_tensors, labels) in enumerate(dataset):
        print("-"*30, str(i), "-"*30)
        logit, embedding_vector = _train_step(sparse_tensors, labels)
        print("[INFO]: embedding_vector:\n", embedding_vector)
        tf_results.append(embedding_vector)

        # FIXME: because plugin sleepd, here is only used for 
        # simulate the same DNN structure. 
        import time
        time.sleep(0.2) # seconds

    return tf_results

Thirdly, define multi-node training loop for SOK.

In [7]:
def test_sok_demo(args, task_id, init_tensors, *random_samples):
    physical_devices = tf.config.list_physical_devices('GPU')
    print("[INFO]: physical_devices on task %d:" %task_id, physical_devices)
    
    port = 12345
    os.environ["TF_CONFIG"] = json.dumps({
        'cluster': {"worker": [args["ips"][i] + ":" + str(port + i) for i in range(args["worker_num"])] },
        'task': {"type": 'worker', "index": task_id}
    })
    strategy = tf.distribute.MultiWorkerMirroredStrategy()
    with strategy.scope():
        sok.Init(global_batch_size=args["global_batch_size"])

        sok_demo = SOKDemo(combiner=args["combiner"], 
                            max_vocabulary_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                            slot_num=args["slot_num"], max_nnz=args["max_nnz"],
                            embedding_vec_size=args["embedding_vec_size"])

        emb_opt = utils.get_embedding_optimizer(args["optimizer"])(learning_rate=0.1)
        dense_opt = utils.get_dense_optimizer(args["optimizer"])(learning_rate=0.1)
    
    sok_saver = sok.Saver()
    sok_saver.load_embedding_values(sok_demo.embedding_layer.embedding_variable, init_tensors)

    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    def _replica_loss(labels, logits):
        loss = loss_fn(labels, logits)
        return tf.nn.compute_average_loss(loss, global_batch_size=args["global_batch_size"])

    @tf.function
    def _train_step(inputs, labels):
        with tf.GradientTape() as tape:
            logit, embedding_vector = sok_demo(inputs, training=True)
            loss = _replica_loss(labels, logit)
        embedding_variables, other_variable = sok.split_embedding_variable_from_others(sok_demo.trainable_variables)
        grads, emb_grads = tape.gradient(loss, [other_variable, embedding_variables])
        if "plugin" not in args["optimizer"]:
            with sok.OptimizerScope(embedding_variables):
                emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                        experimental_aggregate_gradients=False)
        else:
            emb_opt.apply_gradients(zip(emb_grads, embedding_variables),
                                    experimental_aggregate_gradients=False)
        dense_opt.apply_gradients(zip(grads, other_variable))
        return logit, embedding_vector

    sok_results = list()

    def _dataset_fn(input_context):
        replica_batch_size = input_context.get_per_replica_batch_size(args["global_batch_size"])
        dataset = utils.tf_dataset(*random_samples, batchsize=replica_batch_size, to_sparse_tensor=True, repeat=1)
        # because each worker has its own data source, so that no need to shard the dataset.
        return dataset

    dataset = strategy.distribute_datasets_from_function(_dataset_fn)

    for i, (sparse_tensors, replica_labels) in enumerate(dataset):
        print("-" * 30, "step ", str(i), "-" * 30)
        logit, embedding_vector = strategy.run(_train_step, args=(sparse_tensors, replica_labels))
        print("[INFO]: embedding_vector\n", embedding_vector)
        sok_results.append(embedding_vector)
        # FIXME: when the forward computation is too fast, there
        # may exist some conficts with datareader, which cause the program hang.
        import time
        time.sleep(0.2) # seconds

    return sok_results

Fourthly, define subprocess work function to simulate multi-node synchronized training

In [8]:
def compare_sok_with_tf(args, task_id):
    if (args["global_batch_size"] % args["local_gpu_num"] != 0):
        raise ValueError("global_batch_size: %d is not divisible by local_gpu_num: %d"
                            %(args["global_batch_size"], args["local_gpu_num"]))
    if (args["global_batch_size"] % args["worker_num"] != 0):
        raise ValueError("global_batch_size: %d is not divisible by worker_num: %d"
                            %(args["global_batch_size"], args["worker_num"]))

    # each worker generate different dataset
    worker_batch_size = args["global_batch_size"] // args["worker_num"]
    random_samples_local = utils.generate_random_samples(num_of_samples=worker_batch_size * args["iter_num"],
                                                         vocabulary_size=args["local_gpu_num"] * args["max_vocabulary_size_per_gpu"] * args["worker_num"],
                                                         slot_num=args["slot_num"],
                                                         max_nnz=args["max_nnz"])
    utils.save_to_file(r"./random_samples_" + str(task_id) + r".file", *random_samples_local)

    # each worker generate same init tensors, because each worker will do the filtering by itself.
    init_tensors = utils.get_ones_tensor(max_vocab_size_per_gpu=args["max_vocabulary_size_per_gpu"],
                                            embedding_vec_size=args["embedding_vec_size"],
                                            num=args["local_gpu_num"] * args["worker_num"])

    sok_results_local = test_sok_demo(args, task_id, init_tensors, *random_samples_local)
    # save the forward embedding vector from different worker to file
    utils.save_to_file(r"./sok_embedding_vectors_" + str(task_id) + r".file", *sok_results_local)

    # aggregate dataset from different worker
    dataset_filenames = [r"./random_samples_" + str(task_id) + r".file"
                         for task_id in range(args["worker_num"])]
    random_samples_total = [list() for _ in range(args["iter_num"])]
    random_labels_total = [list() for _ in range(args["iter_num"])]
    local_batch_size = args["global_batch_size"] // args["worker_num"]
    for work_id in range(args["worker_num"]):
        samples, labels = utils.restore_from_file(dataset_filenames[work_id])
        for i in range(args["iter_num"]):
            random_samples_total[i].extend(samples[i * local_batch_size : (i + 1) * local_batch_size])
            random_labels_total[i].extend(labels[i * local_batch_size : (i + 1) * local_batch_size])
    random_samples_total = np.concatenate(random_samples_total, axis=0)
    random_labels_total = np.concatenate(random_labels_total, axis=0)

    tf_results = test_tf_demo(args, init_tensors, random_samples_total, random_labels_total)

    # aggregate forward embedding vector from different worker
    sok_results_filenames = [r"./sok_embedding_vectors_" + str(task_id) + r".file"
                             for task_id in range(args["worker_num"])]
    sok_results_total = list()
    for file_name in sok_results_filenames:
        sok_results_local = utils.restore_from_file(file_name)
        sok_results_total.append(sok_results_local)

    if (len(sok_results_total[0]) != len(tf_results)):
        raise ValueError("The length of results obtained from sok: %d is not equal to that of tensorflow: %d."
                        %(len(sok_results_total[0]), len(tf_results)))
    if (len(tf_results) != args["iter_num"]):
        raise ValueError("The length of embedding vectors: %d is not equal to iteration number: %d."
                         %(len(tf_results), args["iter_num"]))

    # for i, sok_vector in enumerate(sok_results_total):
    for i in range(args["iter_num"]):
        if args["local_gpu_num"] != 1:
            sok_vector = tf.concat([tf.concat(sok_results_total[task_id][i].values, axis=0)
                                    for task_id in range(args["worker_num"])], axis=0)
        else:
            sok_vector = tf.concat([sok_results_total[task_id][i]
                                    for task_id in range(args["worker_num"])], axis=0)
        tf.debugging.assert_near(tf.reshape(sok_vector, 
                                            shape=[-1, tf.shape(sok_vector)[-1]]),
                                 tf_results[i],
                                 atol=1e-4,
                                 rtol=1e-4)

    print(("\n[INFO]: With MultiWorkerMirroredStrategy, when %d GPUs are used for each node and %d GPUs in total, "
           "the embedding vectors obtained from TensorFlow and SOK are consistent for %d iterations")
          %(args["local_gpu_num"], args["local_gpu_num"] * args["worker_num"], args["iter_num"]))

Fifthly, create sub CPU processes to simulate multi-node synchronized training 

In [9]:
from multiprocessing import Process

processes = list()
for task_id in range(args["worker_num"]):
    available_gpus = ",".join([str(per_worker_gpu_num * task_id + i)
                              for i in range(per_worker_gpu_num)])
    print("[INFO]: on task: %d, its avaiable GPUs are: %s" %(task_id, available_gpus))
    
    os.environ["CUDA_VISIBLE_DEVICES"] = available_gpus
    process = Process(target=compare_sok_with_tf, args=(args, task_id))
    process.start()
    processes.append(process)
    
    
for process in processes:
    if process.is_alive():
        process.join()

[INFO]: on task: 0, its avaiable GPUs are: 0,1,2,3,4,5,6,7
[INFO]: on task: 1, its avaiable GPUs are: 8,9,10,11,12,13,14,15
[INFO]: begin to generate random samples
[INFO]: begin to generate random samples
[INFO]: generated random samples
[INFO]: generated random samples
[INFO]: dumpped items to file ./random_samples_0.file
[INFO]: dumpped items to file ./random_samples_1.file
[INFO]: physical_devices on task 0: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]
[INFO]: physical_devices on task 1: [PhysicalDevice(nam

}

------------------------------------------------------------  step step   11  ------------------------------------------------------------

[INFO]: embedding_vector
[INFO]: embedding_vector
  PerReplica:{
  0: tf.Tensor(
[[0.90112394 0.90112394 0.90112394 ... 0.90102184 0.90102184 0.90102184]
 [0.9010962  0.9010962  0.9010962  ... 0.9012493  0.9012493  0.9012493 ]
 [0.9010795  0.9010795  0.9010795  ... 0.90097725 0.90097725 0.90097725]
 ...
 [0.90110785 0.90110785 0.90110785 ... 0.90119267 0.90119267 0.90119267]
 [0.9010775  0.9010775  0.9010775  ... 0.90110064 0.90110064 0.90110064]
 [0.9011415  0.9011415  0.9011415  ... 0.90099823 0.90099823 0.90099823]], shape=(4096, 40), dtype=float32),
  1: tf.Tensor(
[[0.9010893  0.9010893  0.9010893  ... 0.90109587 0.90109587 0.90109587]
 [0.901051   0.901051   0.901051   ... 0.9008249  0.9008249  0.9008249 ]
 [0.901079   0.901079   0.901079   ... 0.900967   0.900967   0.900967  ]
 ...
 [0.9012722  0.9012722  0.9012722  ... 0.9009889  0.90098

------------------------------------------------------------  step step   33  ------------------------------------------------------------

[INFO]: embedding_vector
[INFO]: embedding_vector
  PerReplica:{
  0: tf.Tensor(
[[0.704871   0.704871   0.704871   ... 0.7038018  0.7038018  0.7038018 ]
 [0.70504844 0.70504844 0.70504844 ... 0.70853865 0.70853865 0.70853865]
 [0.7068519  0.7068519  0.7068519  ... 0.70444393 0.70444393 0.70444393]
 ...
 [0.7054967  0.7054967  0.7054967  ... 0.70610404 0.70610404 0.70610404]
 [0.7049196  0.7049196  0.7049196  ... 0.7071366  0.7071366  0.7071366 ]
 [0.7025713  0.7025713  0.7025713  ... 0.7045839  0.7045839  0.7045839 ]], shape=(4096, 40), dtype=float32),
  1: tf.Tensor(
[[0.70521504 0.70521504 0.70521504 ... 0.70445836 0.70445836 0.70445836]
 [0.7085757  0.7085757  0.7085757  ... 0.7083036  0.7083036  0.7083036 ]
 [0.70290124 0.70290124 0.70290124 ... 0.7059128  0.7059128  0.7059128 ]
 ...
 [0.70643246 0.70643246 0.70643246 ... 0.7033013  0.7033013 

------------------------------------------------------------  step step   55  ------------------------------------------------------------

[INFO]: embedding_vector
[INFO]: embedding_vector
  PerReplica:{
  0: tf.Tensor(
[[0.5122816  0.5122816  0.5122816  ... 0.5092242  0.5092242  0.5092242 ]
 [0.5185317  0.5185317  0.5185317  ... 0.5199945  0.5199945  0.5199945 ]
 [0.51819414 0.51819414 0.51819414 ... 0.5085467  0.5085467  0.5085467 ]
 ...
 [0.51714337 0.51714337 0.51714337 ... 0.5089298  0.5089298  0.5089298 ]
 [0.51046157 0.51046157 0.51046157 ... 0.5192856  0.5192856  0.5192856 ]
 [0.51097524 0.51097524 0.51097524 ... 0.5158346  0.5158346  0.5158346 ]], shape=(4096, 40), dtype=float32),
  1: tf.Tensor(
[[0.5137605  0.5137605  0.5137605  ... 0.51073706 0.51073706 0.51073706]
 [0.5101888  0.5101888  0.5101888  ... 0.5171164  0.5171164  0.5171164 ]
 [0.511326   0.511326   0.511326   ... 0.51996773 0.51996773 0.51996773]
 ...
 [0.5091344  0.5091344  0.5091344  ... 0.5262461  0.5262461 

------------------------------------------------------------  step step   77  ------------------------------------------------------------

[INFO]: embedding_vector
[INFO]: embedding_vector
  PerReplica:{
  0: tf.Tensor(
[[0.3593893  0.3593893  0.3593893  ... 0.32317182 0.32317182 0.32317182]
 [0.33919325 0.33919325 0.33919325 ... 0.33115053 0.33115053 0.33115053]
 [0.32679847 0.32679847 0.32679847 ... 0.34565246 0.34565246 0.34565246]
 ...
 [0.35293257 0.35293257 0.35293257 ... 0.3350122  0.3350122  0.3350122 ]
 [0.335076   0.335076   0.335076   ... 0.34141397 0.34141397 0.34141397]
 [0.33428544 0.33428544 0.33428544 ... 0.35380754 0.35380754 0.35380754]], shape=(4096, 40), dtype=float32),
  1: tf.Tensor(
[[0.33319855 0.33319855 0.33319855 ... 0.333704   0.333704   0.333704  ]
 [0.32769334 0.32769334 0.32769334 ... 0.34162462 0.34162462 0.34162462]
 [0.3408941  0.3408941  0.3408941  ... 0.34208474 0.34208474 0.34208474]
 ...
 [0.33087453 0.33087453 0.33087453 ... 0.3309713  0.3309713 

...

 [0.32692277 0.32692277 0.32692277 ... 0.32054782 0.32054782 0.32054782]], shape=(65536, 40), dtype=float32)
------------------------------ 7 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[0.35938388 0.35938388 0.35938388 ... 0.32316637 0.32316637 0.32316637]
 [0.33918768 0.33918768 0.33918768 ... 0.33114496 0.33114496 0.33114496]
 [0.3267928  0.3267928  0.3267928  ... 0.34564686 0.34564686 0.34564686]
 ...
 [0.3233582  0.3233582  0.3233582  ... 0.32973903 0.32973903 0.32973903]
 [0.3314612  0.3314612  0.3314612  ... 0.33581683 0.33581683 0.33581683]
 [0.32692277 0.32692277 0.32692277 ... 0.32054782 0.32054782 0.32054782]], shape=(65536, 40), dtype=float32)
------------------------------ 8 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[0.24688369 0.24688369 0.24688369 ... 0.26600617 0.26600617 0.26600617]
 [0.2480166  0.2480166  0.2480166  ... 0.2514321  0.2514321  0.2514321 ]
 [0.24435692 0.24435692 0.24435692 ... 0.24600014 0.24600014 0

  -0.1519821 ]], shape=(65536, 40), dtype=float32)
------------------------------ 14 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.1689989  -0.1689989  -0.1689989  ... -0.10777268 -0.10777268
  -0.10777268]
 [-0.13230377 -0.13230377 -0.13230377 ... -0.11240527 -0.11240527
  -0.11240527]
 [-0.12588634 -0.12588634 -0.12588634 ... -0.11294188 -0.11294188
  -0.11294188]
 ...
 [-0.12267219 -0.12267219 -0.12267219 ... -0.10800198 -0.10800198
  -0.10800198]
 [-0.1359811  -0.1359811  -0.1359811  ... -0.13542189 -0.13542189
  -0.13542189]
 [-0.13517365 -0.13517365 -0.13517365 ... -0.1519821  -0.1519821
  -0.1519821 ]], shape=(65536, 40), dtype=float32)
------------------------------ 15 ------------------------------
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.16269596 -0.16269596 -0.16269596 ... -0.15430662 -0.15430662
  -0.15430662]
 [-0.1764975  -0.1764975  -0.1764975  ... -0.17849879 -0.17849879
  -0.17849879]
 [-0.152884   -0.152884   -0.152884   ... -0.16444835 -

  -0.21160543]], shape=(65536, 40), dtype=float32)
------------------------------ 27 ------------------------------
[INFO]: embedding_vector:
 ------------------------------ 28 ------------------------------
tf.Tensor(
[[-0.1761609  -0.1761609  -0.1761609  ... -0.23750332 -0.23750332
  -0.23750332]
 [-0.23230386 -0.23230386 -0.23230386 ... -0.17313842 -0.17313842
  -0.17313842]
 [-0.21483089 -0.21483089 -0.21483089 ... -0.17595741 -0.17595741
  -0.17595741]
 ...
 [-0.15503663 -0.15503663 -0.15503663 ... -0.20782913 -0.20782913
  -0.20782913]
 [-0.20479324 -0.20479324 -0.20479324 ... -0.1684965  -0.1684965
  -0.1684965 ]
 [-0.21528238 -0.21528238 -0.21528238 ... -0.21160541 -0.21160541
  -0.21160541]], shape=(65536, 40), dtype=float32)
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.23930222 -0.23930222 -0.23930222 ... -0.16324466 -0.16324466
  -0.16324466]
 [-0.1713374  -0.1713374  -0.1713374  ... -0.17283335 -0.17283335
  -0.17283335]
 [-0.16150628 -0.16150628 -0.16150628 ... -0.17818265 -

  -0.08967178]], shape=(65536, 40), dtype=float32)
[INFO]: embedding_vector:
 tf.Tensor(
[[-0.13579781 -0.13579781 -0.13579781 ... -0.16283527 -0.16283527
  -0.16283527]
 [-0.16079614 -0.16079614 -0.16079614 ... -0.09351777 -0.09351777
  -0.09351777]
 [-0.11941607 -0.11941607 -0.11941607 ... -0.12348752 -0.12348752
  -0.12348752]
 ...
 [-0.07437872 -0.07437872 -0.07437872 ... -0.11933292 -0.11933292
  -0.11933292]
 [-0.14259903 -0.14259903 -0.14259903 ... -0.12376738 -0.12376738
  -0.12376738]
 [-0.12998739 -0.12998739 -0.12998739 ... -0.12171997 -0.12171997
  -0.12171997]], shape=(65536, 40), dtype=float32)
------------------------------ 34 ------------------------------
------------------------------[INFO]: embedding_vector:
  35 ------------------------------
tf.Tensor(
[[-0.13579783 -0.13579783 -0.13579783 ... -0.16283526 -0.16283526
  -0.16283526]
 [-0.16079615 -0.16079615 -0.16079615 ... -0.09351775 -0.09351775
  -0.09351775]
 [-0.11941606 -0.11941606 -0.11941606 ... -0.12348746 

  -0.2244833 ]], shape=(65536, 40), dtype=float32)------------------------------

[INFO]: embedding_vector:
 tf.Tensor(
[[-0.20925497 -0.20925497 -0.20925497 ... -0.18087214 -0.18087214
  -0.18087214]
 [-0.23436086 -0.23436086 -0.23436086 ... -0.21694495 -0.21694495
  -0.21694495]
 [-0.22611482 -0.22611482 -0.22611482 ... -0.2561999  -0.2561999
  -0.2561999 ]
 ...
 [-0.20812303 -0.20812303 -0.20812303 ... -0.23730367 -0.23730367
  -0.23730367]
 [-0.22107518 -0.22107518 -0.22107518 ... -0.19635016 -0.19635016
  -0.19635016]
 [-0.2128504  -0.2128504  -0.2128504  ... -0.29057276 -0.29057276
  -0.29057276]], shape=(65536, 40), dtype=float32)
------------------------------ 47 ------------------------------
[INFO]: embedding_vector:
 ------------------------------ 48 ------------------------------
tf.Tensor(
[[-0.2092549  -0.2092549  -0.2092549  ... -0.18087223 -0.18087223
  -0.18087223]
 [-0.23436081 -0.23436081 -0.23436081 ... -0.21694499 -0.21694499
  -0.21694499]
 [-0.22611476 -0.2261147

If no exceptions and their embedding vectors are totally consistent, then a sentence similar to the following one will be printed.
```shell
"[INFO]: With MultiWorkerMirroredStrategy, when 8 GPUs are used for each node and 16 GPUs in total, the embedding vectors obtained from TensorFlow and SOK are consistent for 50 iterations"
```

## Documents ##
We are writing detailed usage samples and API reference documents, and it will be available in the near future.